In [ ]:
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from framework.utils import *
%matplotlib inline

# Comparing forecasts to FPL API values

The FPL API provides forecasts for the next gameweek. This notebook compares our forecasts to these.

In [ ]:
plt.rcParams["figure.figsize"] = (10, 5)

In [ ]:
our_preds = session.query(PlayerPrediction).all()
fpl_preds = FPLDataFetcher().get_player_summary_data()

In [ ]:
def make_our_df_row(pred):
    df = pd.DataFrame()
    df["id"] = [pred.player_id]
    df["predicted_points"] = [pred.predicted_points]
    df["gameweek"] = [pred.gameweek]
    df["method"] = [pred.method]
    return df

def make_fpl_row(pred):
    df = pd.DataFrame()
    df["id"] = [pred["id"]]
    df["fpl_points"] = [float(pred["ep_next"])]
    df["gameweek"] = [5]
    df["name"] = get_player_name(pred["id"])
    return df

def make_pred_df(preds, kind="ours"):
    if kind == "ours":
        f = make_our_df_row
    elif kind == "fpl":
        f = make_fpl_row
        preds = preds.values()
    else:
        raise ValueError
    return pd.concat((f(pred) for pred in preds))

In [ ]:
df_our_preds = make_pred_df(our_preds, kind="ours")
df_fpl_preds = make_pred_df(fpl_preds, kind="fpl")

df_our_preds = df_our_preds[df_our_preds["gameweek"] == 5]
df_all = df_our_preds.merge(df_fpl_preds, left_on="id", right_on="id")

In [ ]:
sns.distplot(df_all["predicted_points"] -  df_all["fpl_points"], kde=False);

In [ ]:
plt.scatter(df_all["predicted_points"].values, df_all["fpl_points"].values)
plt.xlabel("Expected points (AIrsenal)")
plt.ylabel("Expected points (FPL website)")
x = np.linspace(0., 7.)
std = np.std(df_all["predicted_points"] - df_all["fpl_points"])
plt.plot(x, x, ls="--", c='0.5', lw=5, zorder=0.0, label=r"$y = x$")
plt.plot(x, x + 2*std, ls="--", c='0.7', lw=3, zorder=0.0, label=r"$\pm 2\sigma\;(\sigma = {:.2f})$".format(std))
plt.plot(x, x - 2*std, ls="--", c='0.7', lw=3, zorder=0.0)
plt.legend();